# OpenAI Error Clustering - Clean Pipeline

This notebook performs automated clustering of ALL citation sentiment analysis errors using OpenAI's API.

**Pipeline:**
1. Load ALL errors from CSV
2. Use OpenAI to extract linguistic cues and citation intents
3. Cluster errors by AI-identified patterns
4. Export results for analysis

No over-visualization - just clean, systematic clustering.

In [8]:
# Setup and imports
import pandas as pd
import numpy as np
import json
import time
import os
from collections import Counter
from typing import Dict, List, Optional

# OpenAI setup
from openai import OpenAI

# Use the API key from terminal environment
api_key = os.environ.get('OPENAI_API_KEY')
if not api_key:
    print("❌ No API key found in environment!")
    print("Please run: export OPENAI_API_KEY='your-key-here'")
else:
    print(f"✅ Found API key in environment: {api_key[:15]}...")

client = OpenAI(api_key=api_key)
print("✅ OpenAI client initialized")

✅ Found API key in environment: sk-proj-0Y1BzM6...
✅ OpenAI client initialized


In [9]:
# Load ALL errors from CSV
print("📂 Loading error analysis data...")

df = pd.read_csv('error_analysis_results.csv')
print(f"Total samples: {len(df)}")

# Filter for errors only (misclassified samples)
errors_df = df[df['label'] != df['predicted_label']].copy()
total_errors = len(errors_df)

print(f"\n🎯 PROCESSING ALL {total_errors} ERRORS")
print(f"Error rate: {total_errors / len(df) * 100:.1f}%")

# Add error type column
errors_df['error_type'] = errors_df.apply(lambda row: f"{row['label']} → {row['predicted_label']}", axis=1)

print(f"\nError type distribution:")
error_type_counts = errors_df['error_type'].value_counts()
for error_type, count in error_type_counts.items():
    print(f"  {error_type}: {count} ({count/total_errors*100:.1f}%)")

print(f"\n✅ Ready to analyze ALL {total_errors} errors with OpenAI")

📂 Loading error analysis data...
Total samples: 1596

🎯 PROCESSING ALL 239 ERRORS
Error rate: 15.0%

Error type distribution:
  o → p: 98 (41.0%)
  o → n: 58 (24.3%)
  p → o: 48 (20.1%)
  n → o: 17 (7.1%)
  p → n: 13 (5.4%)
  n → p: 5 (2.1%)

✅ Ready to analyze ALL 239 errors with OpenAI


In [13]:
ANALYSIS_PROMPT_COT = """You are an expert linguistic analyst. Your task is to carefully analyze a citation sentence by first reasoning about its properties and then providing a final classification.

Follow these two steps precisely:

**Step 1: Chain-of-Thought Reasoning**
First, provide a step-by-step analysis inside a `reasoning` JSON object. For each category, briefly explain your decision with evidence from the text.
- Analyze each surface-level cue (hedging, contrastive, etc.) one by one.
- Conclude with your reasoning for the final citation `intent`.

**Step 2: Final Classification**
After your reasoning, provide the final classification inside a `classification` JSON object, following the required 0/1 format and intent string.

Return a single, top-level JSON object with exactly two keys: "reasoning" and "classification".

**EXAMPLE OUTPUT STRUCTURE:**
{
  "reasoning": {
    "hedging": "No hedging words like 'may' or 'seems' are present.",
    "contrastive": "The word 'However' is a clear contrastive marker, indicating a shift in argument.",
    "negative_eval": "The phrase 'fails to achieve' is direct criticism.",
    "intent": "The sentence uses a contrastive word and negative evaluation to point out a flaw in prior work. Therefore, the intent is 'Weakness/Limitation'."
  },
  "classification": {
    "hedging": 0,
    "contrastive": 1,
    "positive_eval": 0,
    "negative_eval": 1,
    "metric": 0,
    "multi_citation": 0,
    "intent": "Weakness/Limitation"
  }
}

---

Now, analyze this citation context:
"{CITATION_SENTENCE}"
"""

# The Updated Python Function
def analyze_citation_with_cot(text: str) -> Optional[Dict]:
    """Analyze citation using a Chain-of-Thought prompt for higher accuracy."""
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",  # Using a more capable model is recommended for CoT
            response_format={"type": "json_object"},
            messages=[
                {"role": "system", "content": "You are a helpful assistant designed to output a structured JSON object containing your reasoning and final classification."},
                {"role": "user", "content": ANALYSIS_PROMPT_COT.format(CITATION_SENTENCE=text)}
            ],
            temperature=0.0,
        )
        
        content = response.choices[0].message.content
        full_result = json.loads(content)
        
        print("🧠 Model Reasoning:")
        reasoning = full_result.get("reasoning", {})
        for key, value in reasoning.items():
            print(f"   - {key.title()}: {value}")
        
        classification = full_result.get("classification", {})
        
        clean_result = {
            'hedging': int(classification.get('hedging', 0)),
            'contrastive': int(classification.get('contrastive', 0)),
            'positive_eval': int(classification.get('positive_eval', 0)),
            'negative_eval': int(classification.get('negative_eval', 0)),
            'metric': int(classification.get('metric', 0)),
            'multi_citation': int(classification.get('multi_citation', 0)),
            'intent': str(classification.get('intent', 'Other'))
        }
        return clean_result
            
    except Exception as e:
        print(f"❌ An error occurred during the API call or parsing: {e}")
        return None

# The Test Harness
test_samples = [
    "This approach builds on the work of Smith et al. (2020) to improve performance.",
    "However, the method proposed by Jones (2019) fails to achieve adequate results.",
    "We compare our BLEU scores against those reported in previous studies (Doe et al., 2021), but our approach might be less generalizable."
]

for i, sample in enumerate(test_samples):
    print(f"\n{'='*20}\n🔍 Test {i+1}: {sample}\n{'='*20}")
    result = analyze_citation_with_cot(sample)
    if result:
        print(f"\n✅ Final Classification: {json.dumps(result, indent=2)}")
    else:
        print(f"\n❌ Failed to get a result.")


🔍 Test 1: This approach builds on the work of Smith et al. (2020) to improve performance.
❌ An error occurred during the API call or parsing: '\n  "reasoning"'

❌ Failed to get a result.

🔍 Test 2: However, the method proposed by Jones (2019) fails to achieve adequate results.
❌ An error occurred during the API call or parsing: '\n  "reasoning"'

❌ Failed to get a result.

🔍 Test 3: We compare our BLEU scores against those reported in previous studies (Doe et al., 2021), but our approach might be less generalizable.
❌ An error occurred during the API call or parsing: '\n  "reasoning"'

❌ Failed to get a result.


In [7]:
# Batch process ALL errors with improved error handling
print(f"🚀 PROCESSING ALL {len(errors_df)} ERROR CITATIONS")
print(f"⏱️ Estimated time: {len(errors_df) * 2.0 / 60:.1f} minutes")  # Increased time estimate
print(f"💰 Estimated cost: ~${len(errors_df) * 0.002:.2f}")

# Initialize result columns
errors_df['api_hedging'] = None
errors_df['api_contrastive'] = None
errors_df['api_positive_eval'] = None
errors_df['api_negative_eval'] = None
errors_df['api_metric'] = None
errors_df['api_multi_citation'] = None
errors_df['api_intent'] = None
errors_df['api_success'] = False

# Process each citation with retry logic
successful = 0
failed = 0
consecutive_failures = 0

for idx, row in errors_df.iterrows():
    text = row['text']
    
    # Show progress
    current = idx - errors_df.index[0] + 1
    total = len(errors_df)
    
    if current % 10 == 0 or current == 1:  # More frequent updates
        print(f"📊 Processing {current}/{total} | Success: {successful} | Failed: {failed} | Consecutive fails: {consecutive_failures}")
    
    # Try to analyze citation with retry
    result = None
    max_retries = 2
    
    for attempt in range(max_retries):
        result = analyze_citation(text)
        if result:
            break
        elif attempt < max_retries - 1:
            print(f"  🔄 Retry {attempt + 2} for citation {current}")
            time.sleep(2)  # Longer delay before retry
    
    if result:
        # Store results
        errors_df.at[idx, 'api_hedging'] = result.get('hedging', 0)
        errors_df.at[idx, 'api_contrastive'] = result.get('contrastive', 0)
        errors_df.at[idx, 'api_positive_eval'] = result.get('positive_eval', 0)
        errors_df.at[idx, 'api_negative_eval'] = result.get('negative_eval', 0)
        errors_df.at[idx, 'api_metric'] = result.get('metric', 0)
        errors_df.at[idx, 'api_multi_citation'] = result.get('multi_citation', 0)
        errors_df.at[idx, 'api_intent'] = result.get('intent', 'Other/Unclear')
        errors_df.at[idx, 'api_success'] = True
        successful += 1
        consecutive_failures = 0
    else:
        failed += 1
        consecutive_failures += 1
        
        # If too many consecutive failures, pause longer
        if consecutive_failures >= 5:
            print(f"⚠️ {consecutive_failures} consecutive failures, pausing 10 seconds...")
            time.sleep(10)
            consecutive_failures = 0
    
    # Rate limiting - longer delays due to errors
    time.sleep(1.5)
    
    # Emergency break if failure rate is too high
    if current >= 20 and failed / current > 0.8:
        print(f"❌ STOPPING: Failure rate too high ({failed}/{current} = {failed/current*100:.1f}%)")
        print("Check API key, rate limits, or prompt format")
        break

print(f"\n✅ BATCH PROCESSING COMPLETE!")
print(f"📊 Results: {successful} successful, {failed} failed")
if successful + failed > 0:
    print(f"📈 Success rate: {successful / (successful + failed) * 100:.1f}%")

🚀 PROCESSING ALL 239 ERROR CITATIONS
⏱️ Estimated time: 8.0 minutes
💰 Estimated cost: ~$0.48
📊 Processing 1/239 | Success: 0 | Failed: 0 | Consecutive fails: 0
📊 Processing 20/239 | Success: 4 | Failed: 0 | Consecutive fails: 0
📊 Processing 160/239 | Success: 27 | Failed: 0 | Consecutive fails: 0
📊 Processing 180/239 | Success: 30 | Failed: 0 | Consecutive fails: 0
📊 Processing 300/239 | Success: 50 | Failed: 0 | Consecutive fails: 0
📊 Processing 350/239 | Success: 57 | Failed: 0 | Consecutive fails: 0
📊 Processing 400/239 | Success: 62 | Failed: 0 | Consecutive fails: 0
📊 Processing 440/239 | Success: 75 | Failed: 0 | Consecutive fails: 0
📊 Processing 480/239 | Success: 78 | Failed: 0 | Consecutive fails: 0
📊 Processing 500/239 | Success: 84 | Failed: 0 | Consecutive fails: 0
📊 Processing 550/239 | Success: 93 | Failed: 0 | Consecutive fails: 0
📊 Processing 570/239 | Success: 97 | Failed: 0 | Consecutive fails: 0
📊 Processing 680/239 | Success: 112 | Failed: 0 | Consecutive fails: 0
📊

In [8]:
# Analyze results - focus on successful analyses
successful_results = errors_df[errors_df['api_success'] == True].copy()
print(f"📊 ANALYZING {len(successful_results)} SUCCESSFUL RESULTS")

if len(successful_results) == 0:
    print("❌ No successful results to analyze")
else:
    # Surface cue distribution
    print(f"\n🔧 SURFACE CUE DISTRIBUTION:")
    cue_columns = ['api_hedging', 'api_contrastive', 'api_positive_eval', 'api_negative_eval', 'api_metric', 'api_multi_citation']
    cue_names = ['Hedging', 'Contrastive', 'Positive Eval', 'Negative Eval', 'Metric', 'Multi-Citation']
    
    for col, name in zip(cue_columns, cue_names):
        count = successful_results[col].sum()
        pct = count / len(successful_results) * 100
        print(f"  {name:15}: {count:3d}/{len(successful_results)} ({pct:4.1f}%)")
    
    # Intent distribution
    print(f"\n🎯 INTENT DISTRIBUTION:")
    intent_counts = successful_results['api_intent'].value_counts()
    for intent, count in intent_counts.items():
        pct = count / len(successful_results) * 100
        print(f"  {intent:20}: {count:3d}/{len(successful_results)} ({pct:4.1f}%)")
    
    # Error type breakdown by intent
    print(f"\n❌ ERROR TYPES BY INTENT:")
    for intent in intent_counts.index[:3]:  # Top 3 intents
        intent_data = successful_results[successful_results['api_intent'] == intent]
        error_dist = intent_data['error_type'].value_counts()
        print(f"\n  📂 {intent} ({len(intent_data)} samples):")
        for error_type, count in error_dist.head(3).items():
            pct = count / len(intent_data) * 100
            print(f"    {error_type:20}: {count:2d} ({pct:3.0f}%)")
    
    print(f"\n✅ Analysis complete for {len(successful_results)} citations")

📊 ANALYZING 239 SUCCESSFUL RESULTS

🔧 SURFACE CUE DISTRIBUTION:
  Hedging        :  22/239 ( 9.2%)
  Contrastive    :  38/239 (15.9%)
  Positive Eval  :  14/239 ( 5.9%)
  Negative Eval  :  11/239 ( 4.6%)
  Metric         :  12/239 ( 5.0%)
  Multi-Citation : 202/239 (84.5%)

🎯 INTENT DISTRIBUTION:
  Background          : 239/239 (100.0%)

❌ ERROR TYPES BY INTENT:

  📂 Background (239 samples):
    o → p               : 98 ( 41%)
    o → n               : 58 ( 24%)
    p → o               : 48 ( 20%)

✅ Analysis complete for 239 citations


In [ ]:
# Create clustering signatures and export results
if len(successful_results) > 0:
    print("🔧 CREATING CLUSTERING SIGNATURES...")
    
    # Create surface cue signature
    def create_cue_signature(row):
        cues = []
        if row['api_hedging']: cues.append('H')
        if row['api_contrastive']: cues.append('C')
        if row['api_positive_eval']: cues.append('P+')
        if row['api_negative_eval']: cues.append('P-')
        if row['api_metric']: cues.append('M')
        if row['api_multi_citation']: cues.append('MC')
        return '_'.join(cues) if cues else 'NONE'
    
    successful_results['cue_signature'] = successful_results.apply(create_cue_signature, axis=1)
    successful_results['cluster_label'] = successful_results['api_intent'] + '_' + successful_results['cue_signature']
    
    # Show top clusters
    print(f"\n📂 TOP ERROR CLUSTERS:")
    cluster_counts = successful_results['cluster_label'].value_counts().head(10)
    for cluster, count in cluster_counts.items():
        pct = count / len(successful_results) * 100
        print(f"  {cluster:35}: {count:2d} ({pct:3.1f}%)")
    
    # Export main results
    export_columns = [
        'text', 'label', 'predicted_label', 'error_type',
        'api_hedging', 'api_contrastive', 'api_positive_eval', 'api_negative_eval',
        'api_metric', 'api_multi_citation', 'api_intent',
        'cue_signature', 'cluster_label', 'api_success'
    ]
    
    if 'confidence' in successful_results.columns:
        export_columns.insert(4, 'confidence')
    
    # Export to CSV
    output_file = 'openai_clustered_all_errors.csv'
    successful_results[export_columns].to_csv(output_file, index=False)
    print(f"\n💾 EXPORTED {len(successful_results)} clustered errors to: {output_file}")
    
    # Create cluster summary
    cluster_summary = []
    for cluster_label in cluster_counts.index:
        cluster_data = successful_results[successful_results['cluster_label'] == cluster_label]
        
        # Most common error type in this cluster
        top_error = cluster_data['error_type'].value_counts().iloc[0]
        top_error_type = cluster_data['error_type'].value_counts().index[0]
        
        cluster_summary.append({
            'cluster_label': cluster_label,
            'total_count': len(cluster_data),
            'percentage_of_errors': len(cluster_data) / len(successful_results) * 100,
            'top_error_type': top_error_type,
            'top_error_count': top_error,
            'example_text': cluster_data.iloc[0]['text'][:100] + "..."
        })
    
    cluster_summary_df = pd.DataFrame(cluster_summary)
    summary_file = 'cluster_summary_all_errors.csv'
    cluster_summary_df.to_csv(summary_file, index=False)
    print(f"💾 EXPORTED cluster summary to: {summary_file}")
    
    print(f"\n🎯 CLUSTERING COMPLETE!")
    print(f"📊 Processed: {len(errors_df)} total errors")
    print(f"✅ Successful: {len(successful_results)} analyzed citations")
    print(f"📂 Clusters: {len(cluster_counts)} unique error patterns identified")
    print(f"💾 Files: {output_file}, {summary_file}")

else:
    print("❌ No successful results to cluster")